In [110]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import os, sys
import json
import pickle
from urllib.parse import urlparse
from urllib.request import urlretrieve, unquote
from tqdm.notebook import tqdm

In [8]:
file_path = './data/gender_age_dataset.txt'

In [38]:
df = pd.read_csv(file_path, sep='\t')

In [111]:
df.head(5)

,gender,age,uid,user_json
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,"{""visits"": [{""url"": ""http://zebra-zoya.ru/2000..."
1,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f,"{""visits"": [{""url"": ""http://sweetrading.ru/?p=..."
2,F,25-34,d50237ea-747e-48a2-ba46-d08e71dddfdb,"{""visits"": [{""url"": ""http://ru.oriflame.com/pr..."
3,F,25-34,d502f29f-d57a-46bf-8703-1cb5f8dcdf03,"{""visits"": [{""url"": ""http://translate-tattoo.r..."
4,M,>=55,d503c3b2-a0c2-4f47-bb27-065058c73008,"{""visits"": [{""url"": ""https://mail.rambler.ru/#..."


Просмотр строчки с json

In [40]:
df.iloc[0].user_json

'{"visits": [{"url": "http://zebra-zoya.ru/200028-chehol-organayzer-dlja-macbook-11-grid-it.html?utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun", "timestamp": 1419688144068}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426666298001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426666298000}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426661722001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426661722000}]}'

In [41]:
str1 = json.loads(df.iloc[0].user_json)
str1

{'visits': [{'url': 'http://zebra-zoya.ru/200028-chehol-organayzer-dlja-macbook-11-grid-it.html?utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun',
   'timestamp': 1419688144068},
  {'url': 'http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story',
   'timestamp': 1426666298001},
  {'url': 'http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html',
   'timestamp': 1426666298000},
  {'url': 'http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story',
   'timestamp': 1426661722001},
  {'url': 'http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html',
   'timestamp': 1426661722000}]}

In [138]:
# Используем дальше df_1 для анализа
df_1 = df.iloc[:100]
df_1.head()
# df_1.shape

,gender,age,uid,user_json
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,"{""visits"": [{""url"": ""http://zebra-zoya.ru/2000..."
1,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f,"{""visits"": [{""url"": ""http://sweetrading.ru/?p=..."
2,F,25-34,d50237ea-747e-48a2-ba46-d08e71dddfdb,"{""visits"": [{""url"": ""http://ru.oriflame.com/pr..."
3,F,25-34,d502f29f-d57a-46bf-8703-1cb5f8dcdf03,"{""visits"": [{""url"": ""http://translate-tattoo.r..."
4,M,>=55,d503c3b2-a0c2-4f47-bb27-065058c73008,"{""visits"": [{""url"": ""https://mail.rambler.ru/#..."


In [152]:
df_2 = []
for i in range(100):
    str_i = json.loads(df_1.iloc[i].user_json)
    df_2.append(str_i['visits'][0]) 
df_2

[{'url': 'http://zebra-zoya.ru/200028-chehol-organayzer-dlja-macbook-11-grid-it.html?utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun',
  'timestamp': 1419688144068},
 {'url': 'http://sweetrading.ru/?p=900', 'timestamp': 1419717886224},
 {'url': 'http://ru.oriflame.com/products/product?code=30569',
  'timestamp': 1418840296062},
 {'url': 'http://translate-tattoo.ru/font-selection/?hash=1199c573a5f4da47ed3706b486fa55cc',
  'timestamp': 1418217864467},
 {'url': 'https://mail.rambler.ru/#/folder/', 'timestamp': 1427272415001},
 {'url': 'https://cfire.mail.ru/auth/register/?_1lr=54a015dc6cf29233-1150-2021683_1_6-2021683_1_6',
  'timestamp': 1419777541435},
 {'url': 'http://www.msn.com/ru-ru?pc=up97&ocid=up97dhp',
  'timestamp': 1426704753001},
 {'url': 'http://www.gazprom.ru/press/news/2014/april/article189137/',
  'timestamp': 1419613709992},
 {'url': 'http://lifenews.ru/', 'timestamp': 1427203859001},
 {'url': 'https://www.google.ru/', 'timestamp': 14272377350

In [185]:
a = df_1.user_json.apply(json.loads).values.tolist() 
pd.DataFrame.from_records(a)

df_2 = (
    pd.DataFrame.from_records(a)
    .merge(df_1, left_index=True, right_index = True)
)
df_2.drop(columns = "user_json", inplace = True)
df_2
# df_3 = pd.json_normalize(data=df_2[0])
# df.drop(columns = "fields", inplace = True)

,visits,gender,age,uid
0,[{'url': 'http://zebra-zoya.ru/200028-chehol-o...,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777
1,"[{'url': 'http://sweetrading.ru/?p=900', 'time...",M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f
2,[{'url': 'http://ru.oriflame.com/products/prod...,F,25-34,d50237ea-747e-48a2-ba46-d08e71dddfdb
3,[{'url': 'http://translate-tattoo.ru/font-sele...,F,25-34,d502f29f-d57a-46bf-8703-1cb5f8dcdf03
4,"[{'url': 'https://mail.rambler.ru/#/folder/', ...",M,>=55,d503c3b2-a0c2-4f47-bb27-065058c73008
...,...,...,...,...
95,[{'url': 'http://masterh5.adriver.ru/images/00...,M,35-44,d5731428-7a8b-41e5-a911-fc4e8eb99ee1
96,[{'url': 'http://alfabank.ru/press/news/2014/1...,M,45-54,d573e7aa-54cd-4133-af87-b423a5960565
97,[{'url': 'http://www.mechta.kz/catalog/102/435...,M,18-24,d57566c4-d1d4-4014-ac28-5c8618ab5370
98,"[{'url': 'http://mogilev.gov.by/', 'timestamp'...",F,45-54,d57684f5-7bf1-4345-8cbe-6fdc93629459


In [207]:
# распаковка массива из листа
df_3 = df_2.explode('visits')
df_3

,visits,gender,age,uid
0,{'url': 'http://zebra-zoya.ru/200028-chehol-or...,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777
0,{'url': 'http://news.yandex.ru/yandsearch?cl4u...,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777
0,{'url': 'http://www.sotovik.ru/news/240283-htc...,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777
0,{'url': 'http://news.yandex.ru/yandsearch?cl4u...,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777
0,{'url': 'http://www.sotovik.ru/news/240283-htc...,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777
...,...,...,...,...
99,{'url': 'http://www.the-digital-picture.com/re...,M,18-24,d576d60e-8d3f-4dc3-97a4-3aeb5a097631
99,{'url': 'http://lestinet.com/site/totlist.info...,M,18-24,d576d60e-8d3f-4dc3-97a4-3aeb5a097631
99,{'url': 'http://deita.ru/news/crazy/21.01.2015...,M,18-24,d576d60e-8d3f-4dc3-97a4-3aeb5a097631
99,{'url': 'http://deita.ru/news/crazy/21.01.2015...,M,18-24,d576d60e-8d3f-4dc3-97a4-3aeb5a097631


In [198]:
df_4 = (
    df_3["visits"]
    .apply(pd.Series)[["url","timestamp"]]
)
# df = df.assign(issue_type_name = df_issue_type)

In [205]:
df_4['timestamp']=pd.to_datetime(df_4.timestamp, unit='ms')
df_4.head(7)

,url,timestamp
0,http://zebra-zoya.ru/200028-chehol-organayzer-...,2014-12-27 13:49:04.068
0,http://news.yandex.ru/yandsearch?cl4url=chezas...,2015-03-18 08:11:38.001
0,http://www.sotovik.ru/news/240283-htc-one-m9-z...,2015-03-18 08:11:38.000
0,http://news.yandex.ru/yandsearch?cl4url=chezas...,2015-03-18 06:55:22.001
0,http://www.sotovik.ru/news/240283-htc-one-m9-z...,2015-03-18 06:55:22.000
1,http://sweetrading.ru/?p=900,2014-12-27 22:04:46.224
1,http://sweetrading.ru/?p=884,2014-12-27 22:04:44.437


In [208]:
df_3

,visits,gender,age,uid
0,{'url': 'http://zebra-zoya.ru/200028-chehol-or...,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777
0,{'url': 'http://news.yandex.ru/yandsearch?cl4u...,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777
0,{'url': 'http://www.sotovik.ru/news/240283-htc...,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777
0,{'url': 'http://news.yandex.ru/yandsearch?cl4u...,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777
0,{'url': 'http://www.sotovik.ru/news/240283-htc...,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777
...,...,...,...,...
99,{'url': 'http://www.the-digital-picture.com/re...,M,18-24,d576d60e-8d3f-4dc3-97a4-3aeb5a097631
99,{'url': 'http://lestinet.com/site/totlist.info...,M,18-24,d576d60e-8d3f-4dc3-97a4-3aeb5a097631
99,{'url': 'http://deita.ru/news/crazy/21.01.2015...,M,18-24,d576d60e-8d3f-4dc3-97a4-3aeb5a097631
99,{'url': 'http://deita.ru/news/crazy/21.01.2015...,M,18-24,d576d60e-8d3f-4dc3-97a4-3aeb5a097631


In [234]:
df_5 = df_3.assign(url = df_4['url'], timestamp = df_4['timestamp']).drop(columns='visits')
df_5

,gender,age,uid,url,timestamp
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,http://zebra-zoya.ru/200028-chehol-organayzer-...,2014-12-27 13:49:04.068
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,http://news.yandex.ru/yandsearch?cl4url=chezas...,2015-03-18 08:11:38.001
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,http://www.sotovik.ru/news/240283-htc-one-m9-z...,2015-03-18 08:11:38.000
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,http://news.yandex.ru/yandsearch?cl4url=chezas...,2015-03-18 06:55:22.001
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,http://www.sotovik.ru/news/240283-htc-one-m9-z...,2015-03-18 06:55:22.000
...,...,...,...,...,...
99,M,18-24,d576d60e-8d3f-4dc3-97a4-3aeb5a097631,http://www.the-digital-picture.com/reviews/iso...,2014-12-17 19:59:23.485
99,M,18-24,d576d60e-8d3f-4dc3-97a4-3aeb5a097631,http://lestinet.com/site/totlist.info,2014-12-17 17:40:01.900
99,M,18-24,d576d60e-8d3f-4dc3-97a4-3aeb5a097631,http://deita.ru/news/crazy/21.01.2015/4820243-...,2015-03-23 23:14:42.000
99,M,18-24,d576d60e-8d3f-4dc3-97a4-3aeb5a097631,http://deita.ru/news/crazy/21.01.2015/4820243-...,2015-03-23 23:14:41.001


In [ ]:
def url2domain(url):
    url = re.sub('(http(s)*://)+', 'http://', url)
    parsed_url = urlparse(unquote(url.strip()))
    if parsed_url.scheme not in ['http','https']: return None
    netloc = re.search("(?:www\.)?(.*)", parsed_url.netloc).group(1)
    if netloc is not None: return str(netloc.encode('utf8')).strip()
    return None

In [231]:
# Создаем отдельный столбец с url
df_url = []
# print(range(len(df_5.index)))
# url2domain(df_5.url.iloc[1])
for i in range(len(df_5.index)):
    str_i = url2domain(df_5.url.iloc[i])
    df_url.append(str_i)   

In [235]:
df_5 = df_5.assign(url_domen = df_url)
df_5

,gender,age,uid,url,timestamp,url_domen
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,http://zebra-zoya.ru/200028-chehol-organayzer-...,2014-12-27 13:49:04.068,b'zebra-zoya.ru'
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,http://news.yandex.ru/yandsearch?cl4url=chezas...,2015-03-18 08:11:38.001,b'news.yandex.ru'
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,http://www.sotovik.ru/news/240283-htc-one-m9-z...,2015-03-18 08:11:38.000,b'sotovik.ru'
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,http://news.yandex.ru/yandsearch?cl4url=chezas...,2015-03-18 06:55:22.001,b'news.yandex.ru'
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,http://www.sotovik.ru/news/240283-htc-one-m9-z...,2015-03-18 06:55:22.000,b'sotovik.ru'
...,...,...,...,...,...,...
99,M,18-24,d576d60e-8d3f-4dc3-97a4-3aeb5a097631,http://www.the-digital-picture.com/reviews/iso...,2014-12-17 19:59:23.485,b'the-digital-picture.com'
99,M,18-24,d576d60e-8d3f-4dc3-97a4-3aeb5a097631,http://lestinet.com/site/totlist.info,2014-12-17 17:40:01.900,b'lestinet.com'
99,M,18-24,d576d60e-8d3f-4dc3-97a4-3aeb5a097631,http://deita.ru/news/crazy/21.01.2015/4820243-...,2015-03-23 23:14:42.000,b'deita.ru'
99,M,18-24,d576d60e-8d3f-4dc3-97a4-3aeb5a097631,http://deita.ru/news/crazy/21.01.2015/4820243-...,2015-03-23 23:14:41.001,b'deita.ru'


In [305]:
df_5['url_domen'].str.split('.').apply(lambda x: ' '.join(x))

0               b'zebra-zoya ru'
0              b'news yandex ru'
0                  b'sotovik ru'
0              b'news yandex ru'
0                  b'sotovik ru'
                 ...            
99    b'the-digital-picture com'
99               b'lestinet com'
99                   b'deita ru'
99                   b'deita ru'
99                  b'google ru'
Name: url_domen, Length: 10241, dtype: object

In [306]:
df_5['url_domen_mod']=df_5['url_domen'].str.split('.').apply(lambda x: ' '.join(x))
df_5

,gender,age,uid,url,timestamp,url_domen,url_domen_mod
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,http://zebra-zoya.ru/200028-chehol-organayzer-...,2014-12-27 13:49:04.068,b'zebra-zoya.ru',b'zebra-zoya ru'
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,http://news.yandex.ru/yandsearch?cl4url=chezas...,2015-03-18 08:11:38.001,b'news.yandex.ru',b'news yandex ru'
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,http://www.sotovik.ru/news/240283-htc-one-m9-z...,2015-03-18 08:11:38.000,b'sotovik.ru',b'sotovik ru'
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,http://news.yandex.ru/yandsearch?cl4url=chezas...,2015-03-18 06:55:22.001,b'news.yandex.ru',b'news yandex ru'
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,http://www.sotovik.ru/news/240283-htc-one-m9-z...,2015-03-18 06:55:22.000,b'sotovik.ru',b'sotovik ru'
...,...,...,...,...,...,...,...
99,M,18-24,d576d60e-8d3f-4dc3-97a4-3aeb5a097631,http://www.the-digital-picture.com/reviews/iso...,2014-12-17 19:59:23.485,b'the-digital-picture.com',b'the-digital-picture com'
99,M,18-24,d576d60e-8d3f-4dc3-97a4-3aeb5a097631,http://lestinet.com/site/totlist.info,2014-12-17 17:40:01.900,b'lestinet.com',b'lestinet com'
99,M,18-24,d576d60e-8d3f-4dc3-97a4-3aeb5a097631,http://deita.ru/news/crazy/21.01.2015/4820243-...,2015-03-23 23:14:42.000,b'deita.ru',b'deita ru'
99,M,18-24,d576d60e-8d3f-4dc3-97a4-3aeb5a097631,http://deita.ru/news/crazy/21.01.2015/4820243-...,2015-03-23 23:14:41.001,b'deita.ru',b'deita ru'


In [307]:
from sklearn.feature_extraction.text import CountVectorizer 

In [308]:
CNT = CountVectorizer()

In [315]:
df_5_temp = CNT.fit_transform(df_5['url_domen_mod'])
df_5_temp = pd.DataFrame(df_5_temp.toarray(), columns= CNT.get_feature_names())
df_5_temp

,003ms,10,1001tur,101,110km,1info,1obl,1ori,1prime,22,...,zdrav,zdrawe,zebra,zhivem,znay,zone,zoomby,zoya,zpu,zr
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10236,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10237,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10238,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10239,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [313]:
len(CNT.get_feature_names())

770

In [321]:
df_5_temp.sum().sort_values().tail(10)

ua         285
mail       368
echo       782
msk        782
unian      825
net       1003
avito     1987
smotri    1996
com       2719
ru        5970
dtype: int64

In [252]:
text=df_5[['url_domen']]
text.head(3)

,url_domen
0,b'zebra-zoya.ru'
0,b'news.yandex.ru'
0,b'sotovik.ru'


Предобработка текста

In [238]:
import stop_words
from stop_words import get_stop_words

In [239]:
import pymorphy2
morph=pymorphy2.MorphAnalyzer()

In [240]:
import re
word_re=re.compile('(?u)\\b[^\\W\\d]\\w+\\b')

In [241]:
from nltk.stem.snowball import RussianStemmer
russian_stemmer=RussianStemmer()

In [242]:
from nltk.stem import LancasterStemmer
lancaster_stemmer = LancasterStemmer()

In [244]:
# findall() - ищет все совпадения с шаблоном. Возвращает результирующие строки в виде списка
def pymorphy2_tokenize(s):
    words=re.findall (word_re,s.lower())
    return [morph.parse(w)[0].normal_form for w in words]

In [245]:
def nltk_tokenize(s):
    words=re.findall (word_re,s.lower())
    return [lancaster_stemmer.stem(w) for w in words]

In [247]:
class PyMorphy2Tokenizer(object):
    def __call__(self,doc):
        return pymorphy2_tokenize(doc)

In [248]:
class NltkStemTokenizer(object):
    def __call__(self,doc):
        return nltk_tokenize(doc)

Токенизация

In [246]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [249]:
tfv=TfidfVectorizer(tokenizer=NltkStemTokenizer(), stop_words=stop_words.get_stop_words('ru'), max_features=2000)

In [255]:
text_df=tfv.fit_transform(text.url_domen)

Извлечение именованых сущностей

In [257]:
tokens=tfv.inverse_transform(text_df)
tokens_df=pd.DataFrame([
    {"tokens": list(r)}
    for r in tokens
])

In [303]:
tokens

[array(['ru', 'zoya', 'zebra'], dtype='<U24'),
 array(['yandex', 'news', 'ru'], dtype='<U24'),
 array(['sotovik', 'ru'], dtype='<U24'),
 array(['yandex', 'news', 'ru'], dtype='<U24'),
 array(['sotovik', 'ru'], dtype='<U24'),
 array(['sweetrading', 'ru'], dtype='<U24'),
 array(['sweetrading', 'ru'], dtype='<U24'),
 array(['sweetrading', 'ru'], dtype='<U24'),
 array(['101', 'ru'], dtype='<U24'),
 array(['sweetrading', 'ru'], dtype='<U24'),
 array(['sweetrading', 'ru'], dtype='<U24'),
 array(['sweetrading', 'ru'], dtype='<U24'),
 array(['sweetrading', 'ru'], dtype='<U24'),
 array(['sweetrading', 'ru'], dtype='<U24'),
 array(['sweetrading', 'ru'], dtype='<U24'),
 array(['oesex', 'ru'], dtype='<U24'),
 array(['interfax', 'ru'], dtype='<U24'),
 array(['101', 'ru'], dtype='<U24'),
 array(['interfax', 'ru'], dtype='<U24'),
 array(['euroavtoprokat', 'ru'], dtype='<U24'),
 array(['euroavtoprokat', 'ru'], dtype='<U24'),
 array(['euroavtoprokat', 'ru'], dtype='<U24'),
 array(['euroavtoprokat', 'ru

In [302]:
# tokens[1].tolist()
tokens_1 = []
for i in range(100):
    tokens_1.append(tokens[i].tolist())
tokens_1

[['ru', 'zoya', 'zebra'],
 ['yandex', 'news', 'ru'],
 ['sotovik', 'ru'],
 ['yandex', 'news', 'ru'],
 ['sotovik', 'ru'],
 ['sweetrading', 'ru'],
 ['sweetrading', 'ru'],
 ['sweetrading', 'ru'],
 ['101', 'ru'],
 ['sweetrading', 'ru'],
 ['sweetrading', 'ru'],
 ['sweetrading', 'ru'],
 ['sweetrading', 'ru'],
 ['sweetrading', 'ru'],
 ['sweetrading', 'ru'],
 ['oesex', 'ru'],
 ['interfax', 'ru'],
 ['101', 'ru'],
 ['interfax', 'ru'],
 ['euroavtoprokat', 'ru'],
 ['euroavtoprokat', 'ru'],
 ['euroavtoprokat', 'ru'],
 ['euroavtoprokat', 'ru'],
 ['euroavtoprokat', 'ru'],
 ['eavtoprokat', 'ru'],
 ['wordparts', 'ru'],
 ['rsdn', 'ru'],
 ['euroavtoprokat', 'ru'],
 ['citieslist', 'ru'],
 ['euroavtoprokat', 'ru'],
 ['rutv', 'ru'],
 ['com', 'fodors'],
 ['wheelsabroad', 'com'],
 ['lestinet', 'com'],
 ['help', 'android', 'ru'],
 ['shop', 'dns', 'club', 'ru'],
 ['philips', 'supportforum', 'com'],
 ['philips', 'supportforum', 'com'],
 ['ua', 'rabota'],
 ['enter', 'ru'],
 ['ferra', 'ru'],
 ['ferra', 'ru'],
 ['li

In [296]:
# Анализ морфем
tokens[1]

array(['yandex', 'news', 'ru'], dtype='<U24')

Counter({7: 10241})

In [265]:
from sklearn.cluster import KMeans

In [267]:
kmeans = KMeans(n_jobs=-1)
kmeans.fit(text_df)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=8, n_init=10, n_jobs=-1, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [268]:
result = kmeans.predict(text_df)

In [270]:
text["cluster"] = result
text

/Users/katerinakucheryavenko/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,url_domen,tokens,cluster
0,b'zebra-zoya.ru',"[ru, zoya, zebra]",0
0,b'news.yandex.ru',"[ru, zoya, zebra]",0
0,b'sotovik.ru',"[ru, zoya, zebra]",0
0,b'news.yandex.ru',"[ru, zoya, zebra]",0
0,b'sotovik.ru',"[ru, zoya, zebra]",0
...,...,...,...
99,b'the-digital-picture.com',"[kenwoodworld, com]",0
99,b'lestinet.com',"[kenwoodworld, com]",0
99,b'deita.ru',"[kenwoodworld, com]",0
99,b'deita.ru',"[kenwoodworld, com]",0


In [271]:
for i in set(result):
    print(f"cluster {i}")
    print(text[text.cluster == i]["url_domen"][:5])
    print()

cluster 0
0     b'zebra-zoya.ru'
0    b'news.yandex.ru'
0        b'sotovik.ru'
0    b'news.yandex.ru'
0        b'sotovik.ru'
Name: url_domen, dtype: object

cluster 1
6     b'avito.ru'
6     b'avito.ru'
10    b'avito.ru'
10    b'avito.ru'
10    b'avito.ru'
Name: url_domen, dtype: object

cluster 2
85    b'smotri.com'
85    b'smotri.com'
85    b'smotri.com'
85    b'smotri.com'
85    b'smotri.com'
Name: url_domen, dtype: object

cluster 3
43    b'echo.msk.ru'
43    b'echo.msk.ru'
43    b'echo.msk.ru'
55    b'echo.msk.ru'
55    b'echo.msk.ru'
Name: url_domen, dtype: object

cluster 4
55              b'unian.net'
55              b'unian.net'
55    b'economics.unian.net'
55              b'unian.net'
55              b'unian.net'
Name: url_domen, dtype: object

cluster 5
3     b'translate-tattoo.ru'
17           b'translate.ru'
17           b'translate.ru'
17           b'translate.ru'
17           b'translate.ru'
Name: url_domen, dtype: object

cluster 6
1         b'rabota.ua'
10       b'smac

In [264]:
# df_6 = df_5.assign(tokens = tokens_df)
# df_6.head(10)
# мэтчит по индексу, что является неверным

,gender,age,uid,url,timestamp,url_domen,tokens
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,http://zebra-zoya.ru/200028-chehol-organayzer-...,2014-12-27 13:49:04.068,b'zebra-zoya.ru',"[ru, zoya, zebra]"
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,http://news.yandex.ru/yandsearch?cl4url=chezas...,2015-03-18 08:11:38.001,b'news.yandex.ru',"[ru, zoya, zebra]"
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,http://www.sotovik.ru/news/240283-htc-one-m9-z...,2015-03-18 08:11:38.000,b'sotovik.ru',"[ru, zoya, zebra]"
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,http://news.yandex.ru/yandsearch?cl4url=chezas...,2015-03-18 06:55:22.001,b'news.yandex.ru',"[ru, zoya, zebra]"
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,http://www.sotovik.ru/news/240283-htc-one-m9-z...,2015-03-18 06:55:22.000,b'sotovik.ru',"[ru, zoya, zebra]"
1,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f,http://sweetrading.ru/?p=900,2014-12-27 22:04:46.224,b'sweetrading.ru',"[yandex, news, ru]"
1,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f,http://sweetrading.ru/?p=884,2014-12-27 22:04:44.437,b'sweetrading.ru',"[yandex, news, ru]"
1,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f,http://sweetrading.ru/?p=1002,2014-12-27 22:03:36.375,b'sweetrading.ru',"[yandex, news, ru]"
1,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f,http://101.ru/?an=port_channel_mp3,2014-12-27 22:03:24.934,b'101.ru',"[yandex, news, ru]"
1,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f,http://sweetrading.ru/?cat=62,2014-12-27 21:03:14.423,b'sweetrading.ru',"[yandex, news, ru]"


ПРОБЫ ФУНКЦИЙ

In [159]:
# Тестирование извлечения
a = df_1.user_json.apply(json.loads).values.tolist() 
pd.DataFrame.from_records(a)

,visits
0,[{'url': 'http://zebra-zoya.ru/200028-chehol-o...
1,"[{'url': 'http://sweetrading.ru/?p=900', 'time..."
2,[{'url': 'http://ru.oriflame.com/products/prod...
3,[{'url': 'http://translate-tattoo.ru/font-sele...
4,"[{'url': 'https://mail.rambler.ru/#/folder/', ..."
...,...
95,[{'url': 'http://masterh5.adriver.ru/images/00...
96,[{'url': 'http://alfabank.ru/press/news/2014/1...
97,[{'url': 'http://www.mechta.kz/catalog/102/435...
98,"[{'url': 'http://mogilev.gov.by/', 'timestamp'..."


In [61]:
# df_1 = []
# for i in range(100):
#     str_i = json.loads(df.iloc[i])
#     df_1.append(str_i) 
# df_1

In [101]:
df_1 = []
for i in range(100):
    str_i = json.loads(df.iloc[i].user_json)
    df_1.append(str_i['visits'][0]) 
df_1

[{'url': 'http://zebra-zoya.ru/200028-chehol-organayzer-dlja-macbook-11-grid-it.html?utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun',
  'timestamp': 1419688144068},
 {'url': 'http://sweetrading.ru/?p=900', 'timestamp': 1419717886224},
 {'url': 'http://ru.oriflame.com/products/product?code=30569',
  'timestamp': 1418840296062},
 {'url': 'http://translate-tattoo.ru/font-selection/?hash=1199c573a5f4da47ed3706b486fa55cc',
  'timestamp': 1418217864467},
 {'url': 'https://mail.rambler.ru/#/folder/', 'timestamp': 1427272415001},
 {'url': 'https://cfire.mail.ru/auth/register/?_1lr=54a015dc6cf29233-1150-2021683_1_6-2021683_1_6',
  'timestamp': 1419777541435},
 {'url': 'http://www.msn.com/ru-ru?pc=up97&ocid=up97dhp',
  'timestamp': 1426704753001},
 {'url': 'http://www.gazprom.ru/press/news/2014/april/article189137/',
  'timestamp': 1419613709992},
 {'url': 'http://lifenews.ru/', 'timestamp': 1427203859001},
 {'url': 'https://www.google.ru/', 'timestamp': 14272377350

In [102]:
df_2 = pd.json_normalize(df_1, sep = "-")
df_2

,url,timestamp
0,http://zebra-zoya.ru/200028-chehol-organayzer-...,1419688144068
1,http://sweetrading.ru/?p=900,1419717886224
2,http://ru.oriflame.com/products/product?code=3...,1418840296062
3,http://translate-tattoo.ru/font-selection/?has...,1418217864467
4,https://mail.rambler.ru/#/folder/,1427272415001
...,...,...
95,http://masterh5.adriver.ru/images/0003754/0003...,1427140329000
96,http://alfabank.ru/press/news/2014/12/17/34085...,1419771338059
97,http://www.mechta.kz/catalog/102/43555/,1427187277001
98,http://mogilev.gov.by/,1427001814000


In [103]:
df_2['timestamp']=pd.to_datetime(df_2.timestamp, unit='ms')
df_2.head()

,url,timestamp
0,http://zebra-zoya.ru/200028-chehol-organayzer-...,2014-12-27 13:49:04.068
1,http://sweetrading.ru/?p=900,2014-12-27 22:04:46.224
2,http://ru.oriflame.com/products/product?code=3...,2014-12-17 18:18:16.062
3,http://translate-tattoo.ru/font-selection/?has...,2014-12-10 13:24:24.467
4,https://mail.rambler.ru/#/folder/,2015-03-25 08:33:35.001


In [104]:
df_2

,url,timestamp
0,http://zebra-zoya.ru/200028-chehol-organayzer-...,2014-12-27 13:49:04.068
1,http://sweetrading.ru/?p=900,2014-12-27 22:04:46.224
2,http://ru.oriflame.com/products/product?code=3...,2014-12-17 18:18:16.062
3,http://translate-tattoo.ru/font-selection/?has...,2014-12-10 13:24:24.467
4,https://mail.rambler.ru/#/folder/,2015-03-25 08:33:35.001
...,...,...
95,http://masterh5.adriver.ru/images/0003754/0003...,2015-03-23 19:52:09.000
96,http://alfabank.ru/press/news/2014/12/17/34085...,2014-12-28 12:55:38.059
97,http://www.mechta.kz/catalog/102/43555/,2015-03-24 08:54:37.001
98,http://mogilev.gov.by/,2015-03-22 05:23:34.000


In [ ]:
# Идеи
# Попробовать добавить в сет и посмотреть пересечения
